In [ ]:
import re




LINE_PATTERN = re.compile(
    rf"^(?P<last>.*?)\s+Notes{COLON}\s*Phone{COLON}\s*(?P<phone>\S+)\s+Hotel{COLON}\s*(?P<hotel>.*?)(?:\s+Boat{COLON}\s*(?P<boat>\S+))?(?:\s+Email{COLON}\s*(?P<email>\S+))?\s*$"
)


In [1]:
from datetime import datetime
import pandas as pd
usecols = [
            "Booked Resource Name",
            "Date of Use",
            "Contact Name",
            "Contact Mobile",
            "Tickets Booked",
            "Additional Information",
            "Booking Status",
            "Contact Email",
        ]
tourist_orders=pd.read_excel(r"C:\Users\Nan Li\Documents\Coding\auto_tourist_management\downloads\CT_2025-10-25_2025-10-25.xlsx",
                 sheet_name="Things to Do",
                skiprows=5,
                usecols=usecols,
                dtype={"Contact Mobile": "string"},
                parse_dates=["Date of Use"],
                engine="openpyxl",
                 )

c:\Users\Nan Li\Documents\Coding\auto_tourist_management\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
tourist_orders.columns = (
            tourist_orders.columns.astype(str)
            .str.strip()
            .str.replace(r"\s+", "_", regex=True)
            .str.replace(r"[^0-9A-Za-z_]", "", regex=True)
        )
before = len(tourist_orders)
tourist_orders = tourist_orders[tourist_orders["Booking_Status"] != "Cancelled"]
after = len(tourist_orders)

# ✅ 这里直接重置索引，方便后续 itertuples
tourist_orders = tourist_orders.reset_index(drop=True)

In [9]:

for order in tourist_orders.itertuples(index=True,name="Order"):
    activity_date = order.Date_of_Use.date()
    print(activity_date.strftime("a%y%m%d"))

a251025


In [3]:
from selenium import webdriver

from modules.utils.browser_connector import (
    close_all_chrome,
    start_chrome_with_debug,
    connect_to_existing_browser,
)
import os

In [1]:
import time
import shutil
import logging
from pathlib import Path
from typing import Any

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [4]:
driver = connect_to_existing_browser()

✅ 已接管浏览器，下载目录：c:\Users\Nan Li\Documents\Coding\auto_tourist_management\downloads


In [5]:
from modules.crawlers.crawler_utils import (
    safe_click,
    extract_text_safe,
    match_group_name,
)

In [6]:
cards = driver.find_elements(By.CSS_SELECTOR, "div.booking-info")

In [7]:
orders = []

In [14]:
card=cards[5]

In [15]:
text = extract_text_safe(
            card,
            ".//li[p[@class='label' and normalize-space()='No. of participants:']]//div[contains(@class,'valuesColumns')]",
        )

In [16]:
text

'Unknown'

In [17]:
text = extract_text_safe(
            card,
            ".//li[p[@class='label' and normalize-space()='Unit:']]//div[contains(@class,'valuesColumns')]",
        )

In [18]:
text

'Person x 3'

In [22]:
import re
text = re.search(r"[Xx×]\s*(\d+)", text or "")

In [25]:
print(text)

<re.Match object; span=(7, 10), match='x 3'>


In [23]:
text.group(1)

'3'

In [ ]:
driver.execute_script("""
    const suffix = document.querySelector('.condition-range-picker .ant-picker-suffix');
    const picker = document.querySelector('.condition-range-picker');
    if (suffix) suffix.click(); else if (picker) picker.click();
""")
time.sleep(0.3)

js = """
const picker = document.querySelector('.condition-range-picker');
const inputs = picker.querySelectorAll('input');
if (inputs.length < 2) return { ok: false };

function setVal(el, val){
    if (el.hasAttribute('readonly')) el.removeAttribute('readonly');
    const setter = Object.getOwnPropertyDescriptor(HTMLInputElement.prototype, 'value').set;
    setter.call(el, val);
    el.dispatchEvent(new Event('input', { bubbles: true }));
}

setVal(inputs[0], arguments[0]);
setVal(inputs[1], arguments[1]);
document.body.click();
picker.dispatchEvent(new Event('change', { bubbles: true }));

return { start: inputs[0].value, end: inputs[1].value, ok: true };
"""

In [ ]:
driver.execute_script(js, "2025-10-23", "2025-10-23")

In [ ]:
driver.execute_script(
            "const b=document.querySelector('button[data-name=\"search\"]');if(b)b.click();"
        )

In [ ]:
export_btn_selector = (
            "div[role='tabpanel'][aria-hidden='false'] button[data-name='export']"
        )

In [ ]:
btn = driver.find_element(By.CSS_SELECTOR, export_btn_selector)


In [ ]:
btn.is_enabled()

In [ ]:
driver.execute_script("arguments[0].scrollIntoView(true);", btn)

In [ ]:
driver.execute_script("arguments[0].click();", btn)

In [ ]:

# 1️⃣ 清理旧浏览器
close_all_chrome()

# 2️⃣ 启动并接管浏览器
start_chrome_with_debug()
driver = connect_to_existing_browser()

ct = CTDownloader(
    driver=driver,
    start_date="2025-10-23",
    end_date="2025-10-23",
    download_dir=os.path.dirname(__file__)
)


In [ ]:

ct.open_page()
ct.set_date_range()
file_path = ct.export_bookings(filename="ctrip_orders_20251023.xlsx")

In [ ]:
import re
import time
import json
import os
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    StaleElementReferenceException,
    ElementClickInterceptedException,
    NoSuchElementException,
    TimeoutException,
)
from modules.crawlers.base_crawler import BaseCrawler
from modules.crawlers.crawler_utils import (
    safe_click,
    extract_text_safe,
    clean_traveler_name,
    get_group_keywords_map,
)
from modules.output_utils import extract_country_code_from_phone, format_phone_number

In [ ]:
from modules.browser_connector import (
    close_all_chrome,
    start_chrome_with_debug,
    connect_to_existing_browser,
)
from modules.crawlers.crawler_GYG import GYGCrawler
from modules.crawlers.crawler_Viator import ViatorCrawler
from modules.output_utils import (
    format_booking_line,
    save_orders_to_txt,
)

start_date = "2025-10-24"
end_date = "2025-10-24"
if start_date == end_date:
    output_file = f"a{start_date.replace('-', '')[2:]}.txt"
else:
    output_file = (
        f"a{start_date.replace('-', '')[2:]}_a{end_date.replace('-', '')[2:]}.txt"
    )

In [ ]:
start_chrome_with_debug()
driver = connect_to_existing_browser()

In [ ]:
wait = WebDriverWait(driver, 10)
outer_start_input = wait.until(
    EC.element_to_be_clickable(
        (
            By.XPATH,
            "//label[contains(.,'Participation date')]/following::input[@placeholder='Start date']",
        )
    )
)
outer_start_input.click()
time.sleep(0.3)

popup_start_input = wait.until(
    EC.presence_of_element_located(
        (
            By.XPATH,
            "//input[contains(@class,'ant-calendar-input') and @placeholder='Start date']",
        )
    )
)
popup_end_input = wait.until(
    EC.presence_of_element_located(
        (
            By.XPATH,
            "//input[contains(@class,'ant-calendar-input') and @placeholder='End date']",
        )
    )
)

popup_start_input.send_keys(Keys.CONTROL, "a")
popup_start_input.send_keys(start_date)
popup_end_input.send_keys(Keys.CONTROL, "a")
popup_end_input.send_keys(end_date)
time.sleep(0.3)

search_button = wait.until(
    EC.element_to_be_clickable((By.XPATH, "//button[span[text()='Search']]"))
)
driver.execute_script("arguments[0].click();", search_button)

In [ ]:
wait = WebDriverWait(driver, 10)
xpath = "//div[@role='tab' and normalize-space(text())='Confirmed']"
confirmed_tab = wait.until(
    EC.element_to_be_clickable((By.XPATH, xpath))
)
safe_click(driver,confirmed_tab)


In [ ]:
xpath = "(//span[contains(@class,'i-icon-icon-view-off')])"
eye_icon = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, xpath))
)
# 检查是否真的可见
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, xpath))
)
safe_click(driver, eye_icon)

In [ ]:
cards = driver.find_elements(By.CSS_SELECTOR, "div.booking-info")
orders = []
print(len(cards))

In [ ]:
card=cards[0]

In [ ]:
print(card)

In [ ]:
extract_text_safe(
    card,
    f".//li[p[@class='label' and normalize-space()='Full name:']]//div[contains(@class,'valuesColumns')]"
)

In [ ]:
from modules.browser_connector import (
    close_all_chrome,
    start_chrome_with_debug,
    connect_to_existing_browser,
)
from modules.crawlers.crawler_GYG import GYGCrawler
from modules.crawlers.crawler_Viator import ViatorCrawler
from modules.output_utils import (
    format_booking_line,
    save_orders_to_txt,
)

start_date = "2025-10-23"
end_date = "2025-10-24"
if start_date == end_date:
    output_file = f"a{start_date.replace('-', '')[2:]}.txt"
else:
    output_file = (
        f"a{start_date.replace('-', '')[2:]}_a{end_date.replace('-', '')[2:]}.txt"
    )
    
# # 1️⃣ 清理旧浏览器
# close_all_chrome()

# # 2️⃣ 启动并接管浏览器
# start_chrome_with_debug()
driver = connect_to_existing_browser()
# crawler = GYGCrawler(driver)
crawler = ViatorCrawler(driver)
crawler.open_page(start_date, end_date)

In [ ]:
driver = connect_to_existing_browser()

In [ ]:
from modules.crawlers.crawler_utils import (
    safe_click,
    extract_text_safe,
    clean_traveler_name,
    get_group_keywords_map,
)

In [ ]:
# modules/crawler_Viator.py
import re
import time
from datetime import datetime
from selenium.webdriver.common.by import By
from modules.crawlers.base_crawler import BaseCrawler

In [ ]:
card_btns = driver.find_elements(
    By.XPATH,
    "//button[contains(text(),'Show details') or contains(@data-automation,'show-more')]",
)
for btn in card_btns:
    safe_click(driver, btn)
    time.sleep(0.5)

In [ ]:
phone_btns = driver.find_elements(
    By.XPATH,
    "//button[contains(@class,'PhoneNumberView') and contains(@class,'maskControls')]",
)
for btn in phone_btns:
    safe_click(driver, btn)
    time.sleep(0.5)

In [ ]:
cards = driver.find_elements(
    By.XPATH,
    "//div[contains(@class,'Card__defaultCard') and contains(@class,'BookingSummaryCard__bookingSummaryCard')]",
)
orders = []


In [ ]:
card=cards[3]

In [ ]:
els = card.find_elements(
    By.XPATH,
    ".//span[.//span[contains(text(),'Lead Traveler')]]/span[contains(@class,'LabelAndValue__value')]"
)
print("found elements:", len(els))
if els:
    print("text:", els[0].text)


In [ ]:
msg_btn = card.find_element(
    By.XPATH, ".//button[contains(@data-automation,'message-traveller')]"
)
safe_click(driver, msg_btn)

In [ ]:
driver.find_elements(
    By.XPATH,
    "//button[contains(@class,'PhoneNumberView__maskControls')]",
)

In [ ]:
print("found elements:", len(els))
if els:
    print("text:", els[0].text)

In [ ]:
text="Thursday, October 23rd, 2025 8:10 AM"

In [ ]:
import re

In [ ]:
text = re.sub(r"(\d+)(st|nd|rd|th)", r"\1", text)
print(text)

In [ ]:
parts = text.split(",", 1)[1]
date_str = " ".join(parts.split(" ", 4)[0:4])
print(parts)
print(date_str)

In [ ]:
from datetime import datetime

In [ ]:
dt = datetime.strptime(date_str.strip(), "%B %d, %Y")
print(dt)

In [ ]:
card="""

<div class="Card__defaultCard___H22VE BookingSummaryCard__bookingSummaryCard___Abg7Y"><div class="Card__cardBody___oU0kD"><div style="box-sizing: border-box; position: relative; margin-left: -20px; margin-right: -20px; padding-left: 20px; padding-right: 20px;"><div class="" style="margin-left: -20px; margin-right: -20px; display: flex; flex-wrap: wrap; flex-grow: 0; flex-shrink: 0; align-items: normal; overflow: visible;"><div class="BookingSummaryCard__bookingDateContainer___CHi0G Grid__gridCol___sBQYW" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 0 0 33.3333%; max-width: 33.3333%; margin-left: 0%; right: auto; left: auto;"><span class="BookingSummaryCard__bookingDate___9mai3"><svg viewBox="0 0 26 26" xmlns="http://www.w3.org/2000/svg" class="BookingSummaryCard__calenderIcon___zux-P Icon__icon___YmOY9"><g fill-rule="nonzero"><path d="M23.5 4.5h-3a1 1 0 010-2h4a1 1 0 011 1v21a1 1 0 01-1 1h-23a1 1 0 01-1-1v-21a1 1 0 011-1h4a1 1 0 110 2h-3v19h21v-19z"></path><path d="M5.5 6.5a1 1 0 01-1-1v-4a1 1 0 011-1h3a1 1 0 011 1v4a1 1 0 01-1 1h-3zm1-2h1v-2h-1v2zM17.5 6.5a1 1 0 01-1-1v-4a1 1 0 011-1h3a1 1 0 011 1v4a1 1 0 01-1 1h-3zm1-2h1v-2h-1v2z"></path><path d="M8.5 4.5a1 1 0 110-2h9a1 1 0 010 2h-9zM1.5 9.5a1 1 0 110-2h23a1 1 0 010 2h-23z"></path></g></svg>&nbsp;Thu, Oct 23, 2025</span></div><div class="BookingSummaryCard__statusLabelsContainer___Chsca Grid__gridCol___sBQYW" align="end" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 0 0 66.6667%; max-width: 66.6667%; margin-left: 0%; right: auto; left: auto;"><span class="jumpstart_ui__Badge__badge Badge__badge___vZ39z BookingStatusLabel__statusLabel___xH3lC">Amended</span></div></div><div class="BookingSummaryCard__titleAndMessagesContainer___a0khS" style="margin-left: -20px; margin-right: -20px; display: flex; flex-wrap: wrap; flex-grow: 0; flex-shrink: 0; align-items: normal; overflow: visible;"><div class="BookingSummaryCard__bookingProductSummaryContainer___5L3iV Grid__gridCol___sBQYW" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 0 0 66.6667%; max-width: 66.6667%; margin-left: 0%; right: auto; left: auto;"><div class="BookingSummaryCard__productSummaryContainer___DKoO9"><h3 class="BookingSummaryCard__productTitle___Lonxm">Giethoorn, Afsluitdijk, Zaanse Schans Day Tour Incl Hotel Pick Up</h3><p class="BookingSummaryCard__productSubtitle___SQni0">Giethoorn, Afsluitdijk, Zaanse Schans Day Tour on Mini VIP Bus 08:00</p></div></div><div class="BookingSummaryCard__unreadMessagesContainer___3RNgy Grid__gridCol___sBQYW" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 0 0 33.3333%; max-width: 33.3333%; margin-left: 0%; right: auto; left: auto;"></div></div><hr><div class="" style="margin-left: -20px; margin-right: -20px; display: flex; flex-wrap: wrap; flex-grow: 0; flex-shrink: 0; align-items: normal; overflow: visible;"><div class="Grid__gridCol___sBQYW" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 0 0 50%; max-width: 50%; margin-left: 0%; right: auto; left: auto;"><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Lead Traveler</span></span>: <span class="LabelAndValue__value___IY-eR">Matt Williams</span></span></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ">2 adults</div></div><div class="Grid__gridCol___sBQYW" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 0 0 50%; max-width: 50%; margin-left: 0%; right: auto; left: auto;"><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ BookingSummaryCard__darkGrey___7bTqT"><strong>BR-1326775015</strong></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ">Submitted Oct 16, 2025, 07:22 PM GMT+2</div></div></div><span style="display: table; clear: both;"></span></div><div class=""><div><div><button class="jumpstart_ui__Link__link Button__btnDefault___9bfxo Button__btnLink___Amwxr" data-automation="booking-search-BR-1326775015-show-more" type="button">Hide details <svg viewBox="0 0 8 13" xmlns="http://www.w3.org/2000/svg" class="BookingSummaryCard__seeMoreArrow___I2GHR Arrow__arrowUp___qYPJJ Icon__icon___YmOY9"><path d="M8 6.5L1.28 13 0 11.76 5.44 6.5 0 1.24 1.28 0z" fill-rule="evenodd"></path></svg></button></div></div><div aria-hidden="false" class="rah-static rah-static--height-auto " style="height: auto;"><div style=""><div><div class="BookingSummaryCard__detailsContainer___hpbg7"><div style="box-sizing: border-box; position: relative; margin-left: -20px; margin-right: -20px; padding-left: 20px; padding-right: 20px; max-width: 960px;"><div class="" style="margin-left: -20px; margin-right: -20px; display: flex; flex-wrap: wrap; flex-grow: 0; flex-shrink: 0; align-items: normal; overflow: visible;"><div class="Grid__gridCol___sBQYW" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 1 0 0px; max-width: 100%; margin-left: 0%; right: auto; left: auto;"><h4 class="BookingSummaryCard__detailBookingTypeHeading___CEXCc">Instant Confirmation</h4></div></div><div class="" style="margin-left: -20px; margin-right: -20px; display: flex; flex-wrap: wrap; flex-grow: 0; flex-shrink: 0; align-items: normal; overflow: visible;"><div class="Grid__gridCol___sBQYW" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 0 0 50%; max-width: 50%; margin-left: 0%; right: auto; left: auto;"><div class="BookingSummaryCard__detailValuesContainer___695es"><div class="BookingSummaryCard__maskedPhoneNumber___kIq5X BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__maskedPhoneNumber___kIq5X BookingSummaryCard__darkGrey___7bTqT">Customer phone number</span></span>: <span class="LabelAndValue__value___IY-eR"><div><a href="tel:+1 801-678-6557"><span>  +1 801-678-6557</span></a></div></span></span></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Traveler names</span></span>: <span class="LabelAndValue__value___IY-eR"><ul><li>Matt Williams (Adult)</li><li>Passenger one (Adult)</li></ul></span></span></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Tour language</span></span>: <span class="LabelAndValue__value___IY-eR"><ul><li>English</li></ul></span></span></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Inclusions</span></span>: <span class="LabelAndValue__value___IY-eR"><ul><li>Air-conditioned vehicle</li><li>Bottled water</li><li>Including guided cheese factory tour</li><li>Including guided wooden shoes factory tour</li><li>Including guided diamond demonstration tour</li><li>Including one hour boat cruise in Giethoorn</li><li>All Fees and Taxes</li><li>Entry/Admission - Kooijman Souvenirs &amp; Clogs Wooden Shoe Workshop</li><li>Entry/Admission - Catharina Hoeve Cheese Farm</li><li>Entry/Admission - Zaanse Schans</li></ul></span></span></div></div></div><div class="Grid__gridCol___sBQYW" style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 20px; padding-right: 20px; width: 100%; overflow: visible; flex: 0 0 50%; max-width: 50%; margin-left: 0%; right: auto; left: auto;"><div class="BookingSummaryCard__detailValuesContainer___695es"><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Pickup point</span></span>: <span class="LabelAndValue__value___IY-eR">Motel One Amsterdam-Waterlooplein, Valkenburgerstraat 90, 1011 LZ Amsterdam The Netherlands</span></span></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Booking Source</span></span>: <span class="LabelAndValue__value___IY-eR">Tripadvisor</span></span></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Product code</span></span>: <span class="LabelAndValue__value___IY-eR">311086P1</span></span></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Amendment submitted</span></span>: <span class="LabelAndValue__value___IY-eR">Oct 16, 2025, 08:41 PM GMT+2</span></span></div><div class="BookingSummaryCard__bookingSummaryLabel___bTAMZ LabelAndValue__labelAndValueContainer___JOevi"><span class="LabelAndValue__label___xo+Z8"><span><span class="BookingSummaryCard__darkGrey___7bTqT">Amount you will be paid</span></span>: <span class="LabelAndValue__value___IY-eR">EUR&nbsp;190.92</span></span></div><button class="jumpstart_ui__Link__link Button__btnDefault___9bfxo Button__btnLink___Amwxr" data-automation="booking-search-BR-1326775015-contact-us" type="button">Contact customer care</button><button class="jumpstart_ui__Link__link Button__btnDefault___9bfxo BookingDetailsCtaSection__detailCallToAction___cZrDN Button__btnLink___Amwxr" data-automation="booking-search-BR-1326775015-history" type="button">Booking history</button><button class="jumpstart_ui__Link__link Button__btnDefault___9bfxo BookingDetailsCtaSection__detailCallToAction___cZrDN Button__btnLink___Amwxr" data-automation="booking-search-BR-1326775015-voucher" type="button">Print ticket</button></div></div></div><span style="display: table; clear: both;"></span></div></div></div></div></div></div><hr><div class="BookingSummaryCardWithCtaFooter__footer___WqiWa"><div class="BookingSummaryCardWithCtaFooter__bookingCtaButtonLinks___HWjR9"><div class="ButtonContainerHorizontal__btnContainerHorizontal___H1OXf ButtonContainerHorizontal__stackOnMobile___DlAvu"><button class="jumpstart_ui__Link__link Button__btnDefault___9bfxo Button__btnLink___Amwxr" data-automation="booking-search-BR-1326775015-acknowledge" type="button">Acknowledge</button><button class="jumpstart_ui__Link__link Button__btnDefault___9bfxo Button__btnLink___Amwxr" data-automation="booking-search-BR-1326775015-request-cancelation" type="button">Cancel booking</button></div></div><div class="BookingSummaryCardWithCtaFooter__bookingCtaButtons___w2LvW"><div class="ButtonContainerHorizontal__btnContainerHorizontal___H1OXf ButtonContainerHorizontal__stackOnMobile___DlAvu"><button class="jumpstart_ui__Button__default Button__btnDefault___9bfxo jumpstart_ui__Button__md" data-automation="booking-search-BR-1326775015-message-traveller" type="button"><span>Message traveler</span></button></div></div></div></div></div>"""

In [ ]:
# modules/crawler_KLK.py
import re
import time
import json
import os
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from modules.browser_connector import find_bookmark_url
from modules.output_utils import extract_country_code_from_phone, format_phone_number
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import (
    StaleElementReferenceException,
    ElementClickInterceptedException,
)

# ===============================
# 📌 1. 加载配置
# ===============================
GROUPS_CONFIG_PATH =r"C:\Users\Nan Li\Documents\Coding\auto_tourist_management\config\groups.json"
with open(GROUPS_CONFIG_PATH, "r", encoding="utf-8") as f:
    GROUPS_CONFIG = json.load(f)

KEYWORDS_MAP = GROUPS_CONFIG["keywords"]
# TOGUIDE_MAP = GROUPS_CONFIG["toGuide"]

In [ ]:
def get_value_by_label(block, label_text):
    """
    通用提取函数：
    根据 <p class='label'>label_text</p>，
    找到它所在的 li，然后从 li 内部的 .valuesColumns 取值。
    """
    try:
        li = block.find_element(
            By.XPATH, f".//li[p[@class='label' and normalize-space()='{label_text}']]"
        )
        value_div = li.find_element(
            By.XPATH, ".//div[contains(@class,'valuesColumns')]"
        )
        return value_div.text.strip()
    except Exception as e:
        # 调试用：打印错误，方便定位
        print(f"⚠️ get_value_by_label 失败：{label_text} -> {e}")
        return ""


def get_departure_location(block):
    """
    Departure location 特殊：在 li 下的 span 内
    """
    try:
        li = block.find_element(
            By.XPATH,
            ".//li[p[@class='label' and normalize-space()='Departure location:']]",
        )
        span = li.find_element(By.XPATH, ".//span[1]")
        return span.text.strip()
    except Exception as e:
        print(f"⚠️ get_departure_location 失败：{e}")
        return ""


def extract_traveler_name(booking_info):
    return get_value_by_label(booking_info, "Full name:")


def extract_traveler_phone(booking_info):
    phone = get_value_by_label(booking_info, "Phone number:")
    phone = phone.replace("-", "").replace(" ", "").strip()
    if phone and not phone.startswith("+"):
        phone = f"+{phone}"
    return phone


def extract_traveler_country(phone):
    try:
        return extract_country_code_from_phone(phone) or "XX"
    except:
        return "XX"


def extract_traveler_count(booking_info):
    unit_text = get_value_by_label(booking_info, "Unit:")
    m = re.search(r"[Xx×]\s*(\d+)", unit_text or "")
    return int(m.group(1)) if m else 1


def extract_traveler_hotel(booking_info):
    return get_departure_location(booking_info)


def extract_language(booking_info):
    lang = get_value_by_label(booking_info, "Preferred language:")
    if lang:
        return lang
    pkg = get_value_by_label(booking_info, "Package name:")
    return pkg.split()[0] if pkg else "Unknown"


def extract_group_name(booking_info):
    activity_name = get_value_by_label(booking_info, "Activity name:")
    package_name = get_value_by_label(booking_info, "Package name:")
    if activity_name and package_name:
        return f"{activity_name} - {package_name}"
    return activity_name or package_name or "Unknown"


def extract_activity_date(booking_info):
    t = get_value_by_label(booking_info, "Participation time:")
    m = re.search(r"\d{4}-\d{2}-\d{2}", t or "")
    if not m:
        return None
    date_str = m.group(0)
    try:
        dt = datetime.strptime(date_str, "%Y-%m-%d")
        return dt.strftime("a%y%m%d")
    except:
        return None


In [ ]:
block = """
<div data-v-330d8bc8="" data-v-5cc57b0e="" data-spm-module="Booking_LIST?trg=manual&amp;ext={&quot;objectId&quot;:&quot;ANM200984&quot;,&quot;index&quot;: &quot;1&quot;}" class="booking-item booking-item"><div data-v-330d8bc8="" data-spm-item="__default"><div data-v-330d8bc8="" class="boooking-general-info-operation"><div data-spm-page="BookingList?trg=manual" class="booking-general-info" style="background-color: rgb(250, 250, 250);"><div class="info-left"><img src="https://res.klook.com/image/upload/v1600351703/UED%20Team%EF%BC%88for%20DE%20only%EF%BC%89/category%20icon/L1%20and%20L2/icon_category_attractions_xm_colorful.png" class="info-icon"><span data-spm-module="BookingReferenceIDClick?trg=manual&amp;ext={&quot;obejctId&quot;:&quot;booking_ANM200984&quot;}" class="info-item"><span class="info-title">Booking reference ID: </span><a href="javascript:void(0)" data-spm-item="__default">
        ANM200984
      </a></span><span class="info-item"><span class="info-title">Booking time: </span>
      2025-10-03 11:44:44
      (GMT+8)
    </span><span class="info-item"><span class="info-title">Booking status: </span><span><!----><!----><!----><span class="ant-tag ant-tag-green">Confirmed</span></span><!----><!----><!----></span></div><div class="info-right"><div class="booking-tag"><!----><!----></div><!----></div></div><div data-v-394055da="" class="booking-operation"><div data-v-394055da="" id="BookingList_BookingDetail" data-spm-page="BookingList?trg=manual" class="operation-left"><div data-v-394055da="" class="tag-note"><div data-v-394055da="" class="tag-item" style="background-color: rgb(255, 255, 255); border-color: rgb(217, 217, 217);"></div><span data-v-394055da="" class="tag-edit-text">
          
        </span><span data-v-394055da="" class="tag-edit-text">Tags &amp; notes</span><i data-v-394055da="" aria-label="icon: edit" class="tag-edit-icon anticon anticon-edit"><svg viewBox="64 64 896 896" data-icon="edit" width="1em" height="1em" fill="currentColor" aria-hidden="true" focusable="false" class=""><path d="M257.7 752c2 0 4-.2 6-.5L431.9 722c2-.4 3.9-1.3 5.3-2.8l423.9-423.9a9.96 9.96 0 0 0 0-14.1L694.9 114.9c-1.9-1.9-4.4-2.9-7.1-2.9s-5.2 1-7.1 2.9L256.8 538.8c-1.5 1.5-2.4 3.3-2.8 5.3l-29.5 168.2a33.5 33.5 0 0 0 9.4 29.8c6.6 6.4 14.9 9.9 23.8 9.9zm67.4-174.4L687.8 215l73.3 73.3-362.7 362.6-88.9 15.7 15.6-89zM880 836H144c-17.7 0-32 14.3-32 32v36c0 4.4 3.6 8 8 8h784c4.4 0 8-3.6 8-8v-36c0-17.7-14.3-32-32-32z"></path></svg></i></div><!----><div data-v-394055da="" class="more-menu"><div data-v-394055da="" class="ant-dropdown-link ant-dropdown-trigger"><i data-v-394055da="" aria-label="icon: more" class="anticon anticon-more"><svg viewBox="64 64 896 896" data-icon="more" width="1em" height="1em" fill="currentColor" aria-hidden="true" focusable="false" class=""><path d="M456 231a56 56 0 1 0 112 0 56 56 0 1 0-112 0zm0 280a56 56 0 1 0 112 0 56 56 0 1 0-112 0zm0 280a56 56 0 1 0 112 0 56 56 0 1 0-112 0z"></path></svg></i>
          More
        </div></div></div><div data-v-394055da="" class="operation-right"><!----><!----><div data-v-330d8bc8="" id="BookingList_BookingDetail" data-spm-page="BookingList?trg=manual" class="booking-item-operation" data-v-394055da=""><div data-spm-module="RejectClick?trg=manual&amp;ext={&quot;obejctId&quot;:&quot;booking_ANM200984&quot;}" style="display: inline-block;"><!----></div><div data-spm-module="EditClick?trg=manual&amp;ext={&quot;obejctId&quot;:&quot;booking_ANM200984&quot;}" class="" style="display: inline-block;"><button data-spm-item="__default" type="button" class="confirm-btn ant-btn ant-btn-primary"><span>Edit</span></button></div><!----><!----><!----><!----><!----><div style="max-width: 300px;"><div data-v-00b27b60="" class="craft-booking"></div></div></div></div><div data-v-289993bf="" data-v-394055da=""><!----><!----></div><div data-v-396cf4ea="" data-v-394055da=""><!----></div><!----></div></div><div data-v-330d8bc8="" class="booking-info"><ul data-v-e0d32f78="" data-v-330d8bc8="" class="booking-info-item"><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Activity ID:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">146697</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Activity name:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">Zaanse Schans, Lego Village and Giethoorn Day Tour Incl. Boat Cruise</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Package name:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">【Shared Tour】Price Per Person</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Internal title:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">-</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Cost price amount:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">2,250.00 CNY</div><!----></div></li></ul><ul data-v-e0d32f78="" data-v-330d8bc8="" class="booking-info-item"><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Unit:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">Person x 3</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Participation time:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">2025-10-17 08:00:00</div><!----></div></li></ul><ul data-v-e0d32f78="" data-v-330d8bc8="" class="booking-info-item"><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Voucher type:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">KLOOKCODE</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Confirmation type:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">Instant confirmation</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Confirmation time:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">2025-10-03 11:46:17</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Order channel:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">Klook</div><!----></div></li></ul><ul data-v-e0d32f78="" data-v-330d8bc8="" class="booking-info-item"><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Full name:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">DEVANG PARIKH</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Email:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">devanghparikh@gmail.com</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Phone number:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">91-9892180111</div><!----></div></li></ul><ul data-v-e0d32f78="" data-v-330d8bc8="" class="booking-info-item"><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Preferred language:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><div data-v-4f69d677="" class="valuesColumns">English</div><!----></div></li><li data-v-e0d32f78=""><p data-v-e0d32f78="" class="label">Departure location:</p><div data-v-4f69d677="" data-v-e0d32f78="" class="pick-up-item-container value"><span data-v-4f69d677="">
    Amsterdam Centraal
  </span><span data-v-4f69d677="" class="item-actions"><div data-v-ebe011e4="" data-v-4f69d677="" class="duplication-container item-action" style="font-size: 12px; font-weight: 400;"><svg data-v-7bfe6861="" data-v-ebe011e4="" class="klook-svg klook-svg_copy" style="width: 16px; height: 16px;"><use data-v-7bfe6861="" xlink:href="#copy"></use></svg>Copy Location
  </div><div data-v-4f69d677="" class="item-action" style="font-size: 12px; font-weight: 400;"><svg data-v-7bfe6861="" data-v-4f69d677="" class="klook-svg klook-svg_view-on-map" style="width: 16px; height: 16px;"><use data-v-7bfe6861="" xlink:href="#view-on-map"></use></svg>View on map
      <div data-v-16a54a50="" data-v-4f69d677="" class="map-container item-action-map" style="display: none;"><div data-v-16a54a50="" class="map-instance"></div><!----></div></div></span></div></li></ul></div><div data-v-330d8bc8="" class="view-more"><button data-v-330d8bc8="" type="button" class="ant-btn ant-btn-link"><span>Show more</span><svg data-v-7bfe6861="" data-v-330d8bc8="" class="klook-svg klook-svg_page-down" style="width: 16px; height: 16px;"><use data-v-7bfe6861="" xlink:href="#page-down"></use></svg></button><!----></div></div></div>
      """

In [ ]:
extract_traveler_name(block)

In [ ]:
extract_activity_date(block)

In [ ]:
extract_traveler_count(block)

In [ ]:
extract_traveler_hotel(block)

In [ ]:
import pandas as pd
import re
from datetime import datetime
# ====== 导游信息 ======
guide_info = {
    "PT": {
        "name": "Pete",
        "phone": "+31 6 38 68 38 39",
        "plate": "ZV-653-G",
        "car": "Black Mercedes van",
        "lang": "EN",
    },
    "RE": {
        "name": "Reinier",
        "phone": "+31 6 18 01 07 47",
        "plate": "G-746-NF",
        "car": "Black Mercedes van",
        "lang": "EN",
    },
    "LZ": {
        "name": "Leidse",
        "phone": "+31 6 41 32 32 64",
        "plate": "NK-125-S",
        "car": "Black Mercedes van",
        "lang": "EN",
    },
    "KA": {
        "name": "Kai",
        "phone": "+31 6 29 17 27 81",
        "plate": "R-278-JT",
        "car": "Black Mercedes van",
        "lang": "EN",
    },
    "LY": {
        "name": "Eric",
        "phone": "+31 6 38 37 65 77",
        "plate": "X-010-ZF",
        "car": "Black Mercedes van",
        "lang": "EN",
    },
    "SM": {
        "name": "Simon",
        "phone": "+31 6 47 05 07 46",
        "plate": "T-278-JF",
        "car": "Brown Mercedes van",
        "lang": "EN",
    },
    "SR": {
        "name": "苏瑞",
        "phone": "+31 6 51 22 21 38",
        "plate": "G-746-NF",
        "car": "黑色奔驰商务车",
        "lang": "CN",
    },
    "KD": {
        "name": "邓导, Kendall",
        "phone": "+31 6 86 05 77 66",
        "plate": "KB-585-F",
        "car": "银色奔驰商务车",
        "lang": "CN",
    },
    "Unknown": {
        "name": "Unknown",
        "phone": "N/A",
        "plate": "N/A",
        "car": "Mercedes van",
        "lang": "EN",
    },
}

# ====== 团代码映射 ======
tour_map = {
    "GAZ": "Zaanse Schans, Afsluitdijk and Giethoorn",
    "GZZ": "Zaanse Schans, Lego Village and Giethoorn",
    "GZ": "Zaanse Schans and Giethoorn",
    "RDD": "Rotterdam, Delft and The Hague",
    "RDD-D": "Rotterdam, Delft and The Hague",
    "RDD-M": "Rotterdam, Delft and The Hague",
}


In [ ]:
full_name = "a251009 CA 2 Paul Rodie GAZ V1 RE"
note = """HOTEL: Kimpton De Witt By IHG, Nieuwezijds Voorburgwal 5, 1012 RC Amsterdam
BOAT: Yes

T: 8:20
L: 
N: 
O:"""

In [ ]:
parts = full_name.split()
parts

In [ ]:
 # --- 基本解析 ---
tour_date, country, people = parts[0], parts[1], parts[2]
# 判断是否中文团（含 CN 或 CT）
is_cn = "CN" in parts

# 游客姓名提取
if is_cn:
    guest_name = " ".join(parts[3:-4])
else:
    guest_name = " ".join(parts[3:-3])
guest_name

In [ ]:
# 导游代码
guide_code = parts[-1] if parts[-1] in guide_info else "Unknown"
guide = guide_info[guide_code]

# 团名识别
tour_code = next((p for p in parts if p in tour_map), None)
tour_name = tour_map.get(tour_code, "your booked tour")
guide


In [ ]:
def parse_note(note: str) -> dict:
    """
    将 note 按行解析成 {KEY: value} 字典，只取同一行冒号后的内容。
    例如：'HOTEL: Amsterdam Marriott Hotel, ...' → {'HOTEL': 'Amsterdam Marriott Hotel, ...'}
    """
    fields = {}
    for raw in note.splitlines():
        m = re.match(r'^\s*([A-Za-z]+)\s*:\s*(.*)\s*$', raw)
        if m:
            key = m.group(1).upper()
            value = m.group(2).strip()
            fields[key] = value  # 如果同一个 key 多次出现，以最后一次为准
    return fields

In [ ]:
fields      = parse_note(note)
hotel_raw   = fields.get("HOTEL", "")
boat_raw    = fields.get("BOAT", "")
t_raw       = fields.get("T", "")
meeting_raw = fields.get("L", "")
reason_raw  = fields.get("N", "")
other_raw   = fields.get("O", "")

# 只取 T 行的第一个时间（如 T: 8:20 (8:25) → 8:20）
m_time = re.search(r"\b\d{1,2}:\d{2}\b", t_raw)
pickup_time = m_time.group(0) if m_time else ""

In [ ]:
# === 分离酒店信息 ===
hotel_name, hotel_street, hotel_zipcode = "", "", ""
if "," in hotel_raw:
    parts_hotel = [p.strip() for p in hotel_raw.split(",")]
    if len(parts_hotel) >= 3:
        hotel_name = parts_hotel[0]
        hotel_street = parts_hotel[1]
        hotel_zipcode = ", ".join(parts_hotel[2:])
    elif len(parts_hotel) == 2:
        hotel_name = parts_hotel[0]
        hotel_street = parts_hotel[1]
    elif len(parts_hotel) == 1:
        hotel_name = parts_hotel[0]
else:
    hotel_name = hotel_raw.strip()

In [ ]:
# === 模板类型判断 ===
if hotel_raw=="" or "CS" in reason_raw:
    template_type = 3
elif meeting_raw:
    template_type = 2
else:
    template_type = 1

In [ ]:
# === 时间问候 ===
hour = datetime.now().hour
if hour < 12:
    greeting = "Good morning"
elif hour < 18:
    greeting = "Good afternoon"
else:
    greeting = "Good evening"

In [ ]:
import phonenumbers
phone="+17737457726"
num = phonenumbers.parse(phone, None)
region = phonenumbers.region_code_for_number(num)
print(region)

In [ ]:
import numpy as np
a=1
b=2
c=3
for i in np.linspace(1,3,3):
    a=a+1
    b=b+2
    c=c+3
print(a,b,c)
